In [1]:
import json
import pathpy as pp
from utils import Node
import pickle

In [2]:
with open("data/node_tree.pickle", "rb") as f:
    tree = pickle.load(f)

In [3]:
def clean_name(name):
    out = name.replace('\n', '')
    out = out.split('[')[0]
    out = out.split('(')[0]
    out = out.strip()
    return out

In [4]:
def compat_name(directory, name):
    n_try = name
    n = 2
    if name not in directory:
        return name
    else:
        while n_try in directory:
            n_try = name + str(n)
            n += 1
        return n_try

compat_name(['a', 'b', 'b1'], 'b')

'b2'

In [5]:
def walk(node, alias=None, n=pp.Network(directed=False), depth=0):
    e1 = alias or node.name
    children = node.children
    if e1 not in list(n.nodes.keys()):
        n.add_node(e1, depth=depth)
    for child in children:
        # Names must be new (hence function compat_name)
        # otherwise new node is not added, and subsequent children are
        # appended in the wrong place
        e2_comp = compat_name(n.nodes.keys(), clean_name(child.name))
        n.add_node(e2_comp, depth=depth, is_terminal=child.is_terminal)
        n.add_edge(e1, e2_comp)
        assert n is not None
        n = walk(child, alias=e2_comp, n=n, depth=depth+1)
    
    return n

network = walk

n = network(tree['uralic'], n=pp.Network())

In [6]:
nodes = list(n.nodes.keys())
edges = list(n.edges)
len(nodes)

54

In [7]:
res = []
search = 'Northwestern'
for el in edges:
    if search in el[0] or search in el[1]:
        res.append(el)
res

[]

In [8]:
tt = pp.Network()
tt.add_node('a')
tt.add_node('a')
len(tt)

TypeError: object of type 'Network' has no len()

In [9]:
terminal = []

for node, pars in n.nodes.items():
    if pars.get('is_terminal') is True:
        terminal.append(node)

pp.visualisation.plot(n,
                      height=1000,
                      width=1000,
                      node_color = {**{el: "#FF6699" for el in terminal}, **{"Uralic": "#000000"}},
                      node_text = {el: ''.join([c for c in el if not c.isdigit()]) for el in n.nodes},
                      force_alpha=0.2, force_repel=-10)

In [17]:
len(n.nodes)

598

In [67]:
dd.get("Kashmiri2")

'Kashmiri'